# Weather AUS Dataset ETL

In [2]:
# Dependencies and Setup
import pandas as pd
import csv
import numpy as np
import matplotlib
import datetime
import re


## Extract data

In [3]:
# File to load
weatherAUScsv = "resources/weatherAUS.csv"

# Read csv and create Pandas dataframe for cursory analysis
weatherAUSraw = pd.read_csv(weatherAUScsv)
weatherAUSraw.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,1/12/2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2/12/2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,3/12/2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,4/12/2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,5/12/2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


## Cursory ETL analysis
-File contains weather data for cities by date  
-End product will have multiple datatypes; existing file's datatypes need checking  
-Data not present in all columns, will need to manage null entries  
-City names need to be checked; possibility of spelling errors, typos and unexpected characters  
-Need to check weather variables for logical consistency (eg, temperature range, wind direction naming)  

In [4]:
#Check existing datatypes
weatherAUSraw.dtypes


Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [5]:
# Get descriptive stats on amount of entries in columns, min, max etc 
# Saving descriptive stats as dataframe to sort by counts to evaluate columns by abundance of entries

weatherDescribed = weatherAUSraw.describe()
weatherDescribed.sort_values(by='count', axis=1,ascending=False,inplace=True)
weatherDescribed.head()


,MaxTemp,MinTemp,WindSpeed9am,Temp9am,Humidity9am,WindSpeed3pm,Rainfall,Temp3pm,Humidity3pm,WindGustSpeed,Pressure3pm,Pressure9am,Cloud9am,Cloud3pm,Evaporation,Sunshine
count,144199.000000,143975.000000,143693.000000,143693.000000,142806.000000,142398.000000,142199.000000,141851.00000,140953.000000,135197.000000,130432.000000,130395.00000,89572.000000,86102.000000,82670.000000,75625.000000
mean,23.221348,12.194034,14.043426,16.990631,68.880831,18.662657,2.360918,21.68339,51.539116,40.035230,1015.255889,1017.64994,4.447461,4.509930,5.468232,7.611178
std,7.119049,6.398495,8.915375,6.488753,19.029164,8.809800,8.478060,6.93665,20.795902,13.607062,7.037414,7.10653,2.887159,2.720357,4.193704,3.785483
min,-4.800000,-8.500000,0.000000,-7.200000,0.000000,0.000000,0.000000,-5.40000,0.000000,6.000000,977.100000,980.50000,0.000000,0.000000,0.000000,0.000000
25%,17.900000,7.600000,7.000000,12.300000,57.000000,13.000000,0.000000,16.60000,37.000000,31.000000,1010.400000,1012.90000,1.000000,2.000000,2.600000,4.800000


Aiming to drop null values, but to make DF by dropping all rows with any nulls would reduce dataset significantly.  If data loss from dropping nulls is too significant, can retain more data by splitting the dataset into multiple dataframes. Would be more time consuming, will revisit if necessary

In [6]:
# Check city names
print(f"{weatherAUSraw.Location.nunique(), weatherAUSraw.Location.unique(),weatherAUSraw.Location.count()}")  


(49, array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object), 145460)


Names appear legitimate and without obvious typos; capitalisation can be used to add space between words at a later stage for display purposes

In [7]:
# Suspect all entries in dataframe have city and date
# Check number of date entries, check min and max

print(f"{weatherAUSraw.Date.count(), weatherAUSraw.Date.min(),weatherAUSraw.Date.max()}")  


(145460, '1/01/2008', '9/12/2016')


Count for Location and Date match and have the highest counts of all rows

## Convert Data Types

In [44]:
# Copy original DF and start transformations
weatherAUStransformed = weatherAUSraw
weatherAUStransformed['Date']= pd.to_datetime(weatherAUSraw.Date)

# Ensuring Location object is string-type
weatherAUStransformed['Location'] = weatherAUSraw.Location.astype(str)

weatherAUStransformed.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,City
0,2008-01-12,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,Albury
1,2008-02-12,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,Albury
2,2008-03-12,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,Albury
3,2008-04-12,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,Albury
4,2008-05-12,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,Albury


In [10]:
# Regex to add space in Location
cities_spaced = weatherAUStransformed['Location'].str.replace(r"([A-Z])", r" \1").str.strip()
#weatherComplete['Location'].str.replace(r"([A-Z])", r" \1").str.strip()

cities_df = pd.DataFrame({   
                            "City": cities_spaced,                        
                            })

#cities_spaced.unique()
cities_df.City.unique()

weatherAUStransformed['City'] = cities_spaced
weatherAUStransformed['City'].unique()


array(['Albury', 'Badgerys Creek', 'Cobar', 'Coffs Harbour', 'Moree',
       'Newcastle', 'Norah Head', 'Norfolk Island', 'Penrith', 'Richmond',
       'Sydney', 'Sydney Airport', 'Wagga Wagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'Mount Ginini',
       'Ballarat', 'Bendigo', 'Sale', 'Melbourne Airport', 'Melbourne',
       'Mildura', 'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane',
       'Cairns', 'Gold Coast', 'Townsville', 'Adelaide', 'Mount Gambier',
       'Nuriootpa', 'Woomera', 'Albany', 'Witchcliffe', 'Pearce R A A F',
       'Perth Airport', 'Perth', 'Salmon Gums', 'Walpole', 'Hobart',
       'Launceston', 'Alice Springs', 'Darwin', 'Katherine', 'Uluru'],
      dtype=object)

Start by making a Minitab compatible .csv for quick stats and graphs

In [11]:
weatherAUSminitab = weatherAUStransformed.fillna('*')

weatherAUSminitab.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,City
0,2008-01-12,Albury,13.4,22.9,0.6,*,*,W,44,W,...,22,1007.7,1007.1,8,*,16.9,21.8,No,No,Albury
1,2008-02-12,Albury,7.4,25.1,0,*,*,WNW,44,NNW,...,25,1010.6,1007.8,*,*,17.2,24.3,No,No,Albury
2,2008-03-12,Albury,12.9,25.7,0,*,*,WSW,46,W,...,30,1007.6,1008.7,*,2,21,23.2,No,No,Albury
3,2008-04-12,Albury,9.2,28,0,*,*,NE,24,SE,...,16,1017.6,1012.8,*,*,18.1,26.5,No,No,Albury
4,2008-05-12,Albury,17.5,32.3,1,*,*,W,41,ENE,...,33,1010.8,1006,7,8,17.8,29.7,No,No,Albury


In [12]:
# Export finished Minitab dataset for stats analysis
path = "resources\weatherAUSminitab.csv"

weatherAUSminitab.to_csv(path, index=False, header=True)

In [13]:
# Drop columns. Let's see how much data remains when retaining only rows with complete entries for all columns.

weatherComplete = weatherAUStransformed.dropna(axis=0, how='any')

weatherComplete.describe()


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000
mean,13.464770,24.219206,2.130397,5.503135,7.735626,40.877366,15.667228,19.786778,65.874123,49.601985,1017.239505,1014.795580,4.241705,4.326515,18.204961,22.710333
std,6.416689,6.970676,7.014822,3.696282,3.758153,13.335232,8.317005,8.510180,18.513289,20.197040,6.909357,6.870892,2.797162,2.647251,6.567991,6.836543
min,-6.700000,4.100000,0.000000,0.000000,0.000000,9.000000,2.000000,2.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-0.700000,3.700000
25%,8.600000,18.700000,0.000000,2.800000,5.000000,31.000000,9.000000,13.000000,55.000000,35.000000,1012.700000,1010.100000,1.000000,2.000000,13.100000,17.400000
50%,13.200000,23.900000,0.000000,5.000000,8.600000,39.000000,15.000000,19.000000,67.000000,50.000000,1017.200000,1014.700000,5.000000,5.000000,17.800000,22.400000
75%,18.400000,29.700000,0.600000,7.400000,10.700000,48.000000,20.000000,26.000000,79.000000,63.000000,1021.800000,1019.400000,7.000000,7.000000,23.300000,27.900000
max,31.400000,48.100000,206.200000,81.200000,14.500000,124.000000,67.000000,76.000000,100.000000,100.000000,1040.400000,1038.900000,8.000000,9.000000,39.400000,46.100000


In [45]:
# Roughly one-third of the original entries have complete data. Likely enough data for display purposes.

weatherComplete.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,City
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No,Cobar
6050,2009-02-01,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No,Cobar
6052,2009-04-01,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No,Cobar
6053,2009-05-01,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No,Cobar
6054,2009-06-01,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,...,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No,Cobar


In [46]:
# Exporting reduced file without nulls for PowerBI setup

weatherComplete.to_csv("resources\weatherComplete.csv", index=False, header=True)



In [47]:
# Check city names
cityList =  weatherComplete['City'].unique()

# Convert array to list
list = cityList.tolist()

print(f"{weatherComplete.Location.nunique(),cityList,weatherComplete.Location.count()}")  

# Suspect all entries in dataframe have city and date
# Check number of date entries, check min and max

print(f"{weatherComplete.Date.count(), weatherComplete.Date.min(),weatherComplete.Date.max()}")  

(26, array(['Cobar', 'Coffs Harbour', 'Moree', 'Norfolk Island', 'Sydney',
       'Sydney Airport', 'Wagga Wagga', 'Williamtown', 'Canberra', 'Sale',
       'Melbourne Airport', 'Melbourne', 'Mildura', 'Portland',
       'Watsonia', 'Brisbane', 'Cairns', 'Townsville', 'Mount Gambier',
       'Nuriootpa', 'Woomera', 'Perth Airport', 'Perth', 'Hobart',
       'Alice Springs', 'Darwin'], dtype=object), 56420)
(56420, Timestamp('2007-01-11 00:00:00'), Timestamp('2017-12-06 00:00:00'))


As suspected, dropping any nulls is a very significant reduction.  
Cities with complete entries span all states.  
Only 26/49 cities were producing complete data entries.
Date range similar to raw dataset, not a product of changing reporting variables over time.  
Treating as sample population.

## City Latitude and Longditude Data 

File containing Australian city coordinates downloaded for later use in PowerBI  
Source:https://simplemaps.com/data/world-cities

## Check for matches between dataframes

In [19]:
# Check cities in weatherAUS file can be found in downloaded file

# File to load
worldcities = "resources/worldcities.csv"

# Read csv and create Pandas dataframe for cursory analysis
worldCities = pd.read_csv(worldcities)
worldCities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


In [35]:
# Make a dataframe copy with desired columns

worldCitiesCoords = pd.DataFrame({
                                    "longitude": worldCities['lng'],
                                    "latitude": worldCities['lat'],
                                    "city name": worldCities['city'],
                                    "country": worldCities['country']})

worldCitiesCoords.head()



,city name,country,latitude,longitude
0,Tokyo,Japan,35.6897,139.6922
1,Jakarta,Indonesia,-6.2146,106.8451
2,Delhi,India,28.6600,77.2300
3,Mumbai,India,18.9667,72.8333
4,Manila,Philippines,14.5958,120.9772


In [36]:
# Exclude countries not matching Australia 
AUScities = worldCitiesCoords[~(worldCitiesCoords.country != 'Australia')]
AUScities.head()


,city name,country,latitude,longitude
129,Sydney,Australia,-33.8650,151.2094
141,Melbourne,Australia,-37.8136,144.9631
312,Brisbane,Australia,-27.4678,153.0281
378,Perth,Australia,-31.9522,115.8589
538,Adelaide,Australia,-34.9289,138.6011


In [37]:
# Match cities to weather dataset

town_aus= AUScities[AUScities['city name'].isin(list)]

print(town_aus)

           city name    country  latitude  longitude
129           Sydney  Australia  -33.8650   151.2094
141        Melbourne  Australia  -37.8136   144.9631
312         Brisbane  Australia  -27.4678   153.0281
378            Perth  Australia  -31.9522   115.8589
754         Canberra  Australia  -35.2931   149.1269
2365          Hobart  Australia  -42.8806   147.3250
2926      Townsville  Australia  -19.2564   146.8183
3333          Cairns  Australia  -16.9303   145.7703
3618          Darwin  Australia  -12.4381   130.8411
10153  Coffs Harbour  Australia  -30.3022   153.1189
10745    Wagga Wagga  Australia  -35.1189   147.3689
12697        Mildura  Australia  -34.1889   142.1583
13113  Mount Gambier  Australia  -37.8294   140.7828
14412  Alice Springs  Australia  -23.7000   133.8667
17167           Sale  Australia  -38.1000   147.0667
19494       Portland  Australia  -38.3333   141.6000
20248          Moree  Australia  -29.4650   149.8344
23090      Nuriootpa  Australia  -34.4667   13

In [40]:
# Checking the number of towns that will have coordinate data, from the 26 towns found with complete data
print(town_aus['city name'].count())


19


In [43]:
# Exporting reduced dataframe for PowerBI use

output = "town_aus.csv"
town_aus.to_csv(output, index=False, header=True)
